In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn 
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 输入图片是28 * 28
n_inputs = 28      # 输入一行，一行有28个数据
max_time = 28      # 一共28行
lstm_size = 100    # 隐层单元
n_classes = 10     # 10个分类
# 每个批次的大小
batch_size = 50
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))

#初始化偏置
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))


# 定义 RNN 网络
def RNN(X, weights, biases):
    # input = [batch_size, max_time, n_inputs]
    inputs = tf.reshape(X, [-1, max_time, n_inputs])
    # 定义 LSTM 基本CELL
    # lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size)          由于版本不支持
    lstm_cell = rnn.BasicLSTMCell(lstm_size) 
    #final_state[0]是cell state
    #final_state[1]是hidden_state
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype="float")
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    return results


# 计算RNN 的返回结果
prediction = RNN(x, weights, biases)
# 损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
# 使用 AdamOptimizer 进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))  # argmax 返回一维变量中最大值所在位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})

        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter" + str(epoch) + ", Test Accuracy" + str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, Test Accuracy0.6317
Iter1, Test Accuracy0.8026
Iter2, Test Accuracy0.8294
Iter3, Test Accuracy0.8428
Iter4, Test Accuracy0.9097
Iter5, Test Accuracy0.9309


question:
1.module 'tensorflow.contrib.rnn' has no attribute 'core_rnn_cell'
解决方法：
由于版本的问题。1.0版本改了很多
            #原代码是这样的：
            lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size)
            #应该改为：
            from tensorflow.contrib import rnn 

            lstm_cell = rnn.BasicLSTMCell(lstm_size) 